In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('churn').getOrCreate()

### Load data

In [4]:
data = spark.read.csv('data/Logistic_Regression/customer_churn.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                           outputCol='features')

In [10]:
output = assembler.transform(data)

### Take features and churn columns

In [12]:
final_data = output.select(['features', 'churn'])

In [13]:
final_data.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:
model = LogisticRegression(labelCol='churn')

In [19]:
fitted_model = model.fit(train_data)

In [20]:
summary = fitted_model.summary

In [21]:
summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               620|                620|
|   mean|0.1693548387096774|0.13709677419354838|
| stddev|0.3753678689665351| 0.3442272002873788|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



### Evaluate the model on the test data

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [25]:
preds = fitted_model.evaluate(test_data)

In [28]:
preds.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.81161629641926...|[0.99890029861357...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.38435670404089...|[0.99977163951216...|       0.0|
|[29.0,12711.15,0....|    0|[5.85657511106660...|[0.99714713649848...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.43031107490004...|[0.96863851894374...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.44668429290345...|[0.98841835231977...|       0.0|
|[30.0,11575.37,1....|    1|[4.57405356341114...|[0.98978927636381...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.23936457774286...|[0.96228905753511...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.93960333153278...|[0.99289342790280...|       0.0|
|[32.0,8011.38,0.0...|    0|[1.83781255023108...|[0.86268979614478...|       0.0|
|[32.0,11540.86,

In [29]:
churn_eval = BinaryClassificationEvaluator(labelCol = 'churn')

In [31]:
auc = churn_eval.evaluate(preds.predictions)

In [32]:
print(f'AUC: {auc}')

AUC: 0.8733806146572127


### Predict on new data

In [34]:
fitted_model = model.fit(final_data)

In [38]:
new_customers = spark.read.csv('data/Logistic_Regression/new_customers.csv', inferSchema=True, header=True)

In [39]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [51]:
test_new_customers = assembler.transform(new_customers).select(['features', 'company'])

In [52]:
test_new_customers.show()

+--------------------+----------------+
|            features|         company|
+--------------------+----------------+
|[37.0,9935.53,1.0...|        King Ltd|
|[23.0,7526.94,1.0...|   Cannon-Benson|
|[65.0,100.0,1.0,1...|Barron-Robertson|
|[32.0,6487.5,0.0,...|   Sexton-Golden|
|[32.0,13147.71,1....|        Wood LLC|
|[22.0,8445.26,1.0...|   Parks-Robbins|
+--------------------+----------------+



In [53]:
preds = fitted_model.transform(test_new_customers)

In [54]:
preds.show()

+--------------------+----------------+--------------------+--------------------+----------+
|            features|         company|       rawPrediction|         probability|prediction|
+--------------------+----------------+--------------------+--------------------+----------+
|[37.0,9935.53,1.0...|        King Ltd|[2.22168705251440...|[0.90218018099704...|       0.0|
|[23.0,7526.94,1.0...|   Cannon-Benson|[-6.2207530595020...|[0.00198380445828...|       1.0|
|[65.0,100.0,1.0,1...|Barron-Robertson|[-3.7691621189402...|[0.02255110110413...|       1.0|
|[32.0,6487.5,0.0,...|   Sexton-Golden|[-5.0956222016519...|[0.00608622642084...|       1.0|
|[32.0,13147.71,1....|        Wood LLC|[1.10475867224153...|[0.75115067517475...|       0.0|
|[22.0,8445.26,1.0...|   Parks-Robbins|[-1.6896019277064...|[0.15582819767635...|       1.0|
+--------------------+----------------+--------------------+--------------------+----------+



In [55]:
preds.select(['company', 'prediction']).show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

